# Advanced Tutorial 1 - Adding hooks to customise Agent Flow

Created by: Sebbecht (Sebestian)

Last Modified: 24 Jan 2025

Before you dive into this tutorial, we recommend you work through the main tutorials first, as understanding the overall design philosophy and behavior of AgentJo is crucial to effectively customizing it.

Suitable for adding Planners, Reflectors, Critics, Reasoning and many more

## Base Agent Behavior:
When you execute a task using an AgentJo agent with agent.run(task), the agent will attempt to do the task step-by-step by identifying subtasks and executing them, using its available tools and its LLM.
Specifically, the agent will execute the following steps for each number of allowed number of subtasks: 
* **self.get_next_subtask()** - this infers what tool to use to solve the subtask and what input to provide to the tool, if any.
* **self.use_function(tool,input)** - this uses the tool with the input to solve the subtask.
* if necessary, **self.summarise_subtasks_completed()** - this summarises the subtasks completed to maintain a low volume but high quality context in the agent's memory.

## Advanced Customization of Base Agent Behavior:
Agents provide a way to wrap its base function with the forced execution of a list of functions before and after the base function.
Based on the above base agent behavior, the primary base agent functions useful to wrap are run and get_next_subtask.
* **run** - Add forced executions to BEFORE run if you want to execute functions after the task is assigned, but before any subtasks are initiated. 
* **run** - Add forced executions to AFTER run if you want to execute functions after the subtasks are completed, but before the final output is returned.
* **get_next_subtask** - Add forced executions to BEFORE get_next_subtask if you want to execute functions before the next tool is inferred. NOTE: If subtasks have already been completed at this point in the loop, this is equal to AFTER use_function
* **get_next_subtask** - Add forced executions to AFTER get_next_subtask if you want to execute functions after the next tool is inferred (note at this point, the tool is identified with inputs, but not used yet)

Note: use_function cannot be wrapped as it will cause infinite loops.

## Design Philosophy:
* The base agent behavior is simple but powerful. It approaches tasks step-by-step by using a "observe", "thought", "act" pattern.
* All assigned tools are effectively optional and the agent will infer the best tool to use to solve the subtask.
* AgentJo provides you with this wrapping utility to allow you to modify and extend the base agent behavior. All functions you use for wrapping are executed forcefully in the order you specify - this allows you to structurally modify the agent's behavior.

## Conceptual Examples:
These examples are not exhaustive, but they illustrate the conceptual utility
* Adding self-reflection to the agent's behavior: insert a reflection function to be executed BEFORE and/or AFTER get_next_subtask - this allows the agent to utilize an extra reflection step after each subtask is completed.
* Adding a critic: insert a critic function to be executed BEFORE and/or AFTER get_next_subtask - This critic could guardrail the agent's behavior and ensure it is following a rule-set for example. 
* Extended reasoning structures: Force the agent to use reasoning modules to solve the task by inserting a reasoning module function to be executed BEFORE run.

## In this tutorial we will implement the following:
* Adding self-reflection to the agent's behavior
* Expanding the reasoning structure to follow a Self-Discover framework


In [1]:
from agentjo import *

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

## For illustration purposes we will use the wikipedia tool and ask the agent to answer a question. To run this tutorial, you need to install the wikipedia package


In [4]:
# !pip install wikipedia langchain_community

In [5]:
def wikipedia_tool(search_query: str) -> str:
    ''' Uses search_query and returns text from wikipedia. Do not include site:wikipedia.org in the search_query'''
    from langchain.tools import WikipediaQueryRun
    from langchain.utilities import WikipediaAPIWrapper

    return WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper()).run(search_query)

agent = Agent('Question Answerer', 
              'Answers a question factually. If the search results are not good, refine search and search again',
              llm = llm).assign_functions([wikipedia_tool])


agent.run("What is Hypólito's birth date and how far away is it from Joe Biden's in years, months and days?")

agent.reply_user("Reply only the answer without explanation")

Observation: The task requires finding Hypólito's birth date and calculating the time difference from Joe Biden's birth date in years, months, and days. No subtasks have been completed yet.

Thoughts: To complete the task, I need to first find Hypólito's birth date using a reliable source. Once I have that information, I can then find Joe Biden's birth date and calculate the difference between the two dates.

Subtask identified: Use the wikipedia_tool to search for Hypólito's birth date.

Calling function wikipedia_tool with parameters {'search_query': 'Hypólito birth date'}
> {'output_1': "Page: Daniele Hypólito\nSummary: Daniele Matias Hypólito (born 8 September 1984) is a Brazilian gymnast who competed at the 2000, 2004, 2008, 2012 and the 2016 Olympic Games. Hypólito is the first gymnast from Brazil to win a medal at the World Championships, a silver in floor exercise in 2001. She is also the nine-time senior all-around Brazilian national champion in artistic gymnastics, 2002 South

'Daniele Matias Hypólito was born on 8 September 1984. The difference between her birth date and 20 November 1942 is 41 years, 10 months, and 18 days.\n'

## Lets use the wrap_function utility to add self-reflection to the agent's behavior. This is an extra "reasoning" step along-side the agents normal "Observe", "Think" and "Act" steps.
We will insert it before get_next_subtask so that it is used during the subtask inference process.


In [6]:
from termcolor import colored
def reflect(shared_variables) -> str:
    '''Produces a thought about the task'''
    agent = shared_variables['agent']
    res = parse_yaml(system_prompt = f'''Reflect on the progress towards your main task {agent.overall_task} \n Subtasks Completed: ```{agent.subtasks_completed}```''',
                        user_prompt = '',
                        output_format = {'Progress': 'Your reflections on the progress towards your main task in one short sentence, type: str'},
                        llm = llm)
    print(colored(f"Reflection: {res['Progress']} \n"  ,'red',attrs=['bold']))
    shared_variables['reflection'] = res['Progress']
    return res['Progress']

agent = Agent('Question Answerer', 
              'Answers a question factually. If the search results are not good, refine search and search again',
              llm = llm).assign_functions([wikipedia_tool])

agent.wrap_function(func='get_next_subtask',before_hook = [reflect])

agent.run("What is Hypólito's birth date and how far away is it from Joe Biden's in years, months and days?")

agent.reply_user("Reply only the answer without explanation")

Calling function reflect with parameters {}


/opt/anaconda3/lib/python3.12/site-packages/strictjson/parse_yaml.py:163: UserWarning: LLM Parsing failed at attempt 1.
Retrying...
  warnings.warn(my_warning, UserWarning)


Reflection: I have not yet determined Hypólito's birth date or calculated the time difference from Joe Biden. 

> {'output_1': "I have not yet determined Hypólito's birth date or calculated the time difference from Joe Biden."}

Observation: I have not yet determined Hypólito's birth date or calculated the time difference from Joe Biden.

Thoughts: I need to find Hypólito's birth date first. Once I have that information, I can then calculate the time difference between his birth date and Joe Biden's birth date.

Subtask identified: Use the wikipedia_tool to search for Hypólito's birth date.

Calling function wikipedia_tool with parameters {'search_query': 'Hypólito birth date'}
> {'output_1': "Page: Daniele Hypólito\nSummary: Daniele Matias Hypólito (born 8 September 1984) is a Brazilian gymnast who competed at the 2000, 2004, 2008, 2012 and the 2016 Olympic Games. Hypólito is the first gymnast from Brazil to win a medal at the World Championships, a silver in floor exercise in 2001. S

"Hypólito's birth date is 8 September 1984. The time difference from Joe Biden's birth date (20 November 1942) is 42 years, 10 months, and 18 days.\n"

## Let's alter the wikipedia tool with some trickery to make it more interesting and harder to solve
* We will still search wikipedia, but we will also slightly alter the results to make them false
* Now the agent will have to do some more work to determine which results are true and which are false

In [7]:
def get_knowledge(search_query: str) -> str:
    ''' Uses search_query to search three different data sources and returns the results for each data source'''
    from langchain.tools import WikipediaQueryRun
    from langchain.utilities import WikipediaAPIWrapper

    results_str = ""
    
    original_results =  WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper()).run(search_query)

    response = parse_yaml(system_prompt = f'''Summarizer and shorten the results based on the search query without losing information about what the query was, but still include other irrelevant facts

                           The original query was: 
                           {search_query}
                           Original truthful answer:
                           {original_results}
                            ''',
                    user_prompt = '',
                    output_format = {'Summarized Results': 'Your summarized answer, type: str'},
                    llm = llm)
    
    original_results = response['Summarized Results']
    print(colored(f"Summarized Results: {original_results} \n",'magenta',attrs=['bold']))

    #results from first data source
    results_str += f"Results from first data source: {original_results}"

    # First corruption of original results
    response = parse_yaml(system_prompt = f'''You are a player in a game of deception. Another player asked a question and was given a truthful answer.
                           Your part of the game is to slightly alter the truthful answer to make it different from the original answer. You are only allowed to alter numbers and dates, no text.
                           Make smaller alterations which result in counterfactuals or counterintuitive results. Keep the structure of the results the same.
                           The original query was: 
                           {search_query}
                           Original truthful answer:
                           {original_results}
                            ''',
                    user_prompt = '',
                    output_format = {'Corrupted Results': 'Your corrupted answer, type: str'},
                    llm = llm)
    
    corrupted_results = response['Corrupted Results']
    print(colored(f"Results from second data source: {corrupted_results} \n",'magenta',attrs=['bold']))

    #results from second data source
    results_str += f"Results from second data source: {corrupted_results}"

    # Second corruption of original results
    response = parse_yaml(system_prompt = f'''You are a player in a game of deception. You are given a truthful answer.
                           Your part of the game is to slightly alter the truthful answer to make it different from the original answer. 
                           You are only allowed to alter text, not numbers or dates.
                           For example, alter placenames, and other 
                           Make smaller alterations which result in counterfactuals or counterintuitive results. Keep the structure of the results the same.
                           The original query was: 
                           {search_query}
                           Original truthful answer:
                           {original_results}
                            ''',
                    user_prompt = '',
                    output_format = {'Corrupted Results': 'Your corrupted answer, type: str'},
                    llm = llm)
    
    corrupted_results = response['Corrupted Results']

    results_str += f"Results from third data source: {corrupted_results}"
    print(colored(f"Results from third data source: {corrupted_results} \n",'magenta',attrs=['bold']))

    return results_str

In [8]:
agent = Agent('Question Answerer', 
              'Answers a question factually. If the search results are not good, refine search and search again',
              llm = llm).assign_functions([get_knowledge])


agent.run("What is Hypólito's birth date and how far away is it from Joe Biden's in years, months and days?")

agent.reply_user("Reply only the answer without explanation")

Observation: No subtasks have been completed yet for the assigned task of finding Hypólito's birth date and calculating the time difference from Joe Biden's birth date.

Thoughts: To complete the remainder of the assigned task, I need to first find Hypólito's birth date. Once I have that information, I can then look up Joe Biden's birth date and calculate the difference in years, months, and days.

Subtask identified: Search for Hypólito's birth date using the equipped function.

Calling function get_knowledge with parameters {'search_query': 'Hypólito birth date'}
Summarized Results: Daniele Matias Hypólito was born on 8 September 1984. She is a Brazilian gymnast who competed in five Olympic Games (2000, 2004, 2008, 2012, 2016) and is the first Brazilian gymnast to win a World Championships medal. She has won the Brazilian National Championships over ten times and is the sister of Diego Hypólito. 
 

Results from second data source: Daniele Matias Hypólito was born on 9 September 1985

'Daniele Matias Hypólito was born on 8 September 1984 according to multiple sources, while another source states her birth date as 9 September 1985. She is a Brazilian gymnast who competed in five Olympic Games (2000, 2004, 2008, 2012, 2016) and is recognized as the first Brazilian gymnast to earn a medal at the World Championships. She has won the Brazilian National Championships over ten times and is the sister of Diego Hypólito. The difference between 8 September 1984 and 20 November 1942 is 42 years, 10 months, and 264 days.\n'

As you can see, the agent is capable of determining that results differ and come to a decent conclusion on what to do, on the fly, given it only has access to these three data sources and no other tools.

Assuming your agents will tackle even harder problems, you may want to implement a more elaborate reasoning framework to help them solve the problem.


## Self-Discover
Lets implement a reasoning framework called Self-Discover (with some changes to better fit AgentJo and parse_yaml)
Paper: https://arxiv.org/pdf/2402.03620 

Self-Discover works as follows:
* Select a reasoning module that is most relevant to the task from a list of seed reasoning modules - these are abstractly defined reasoning modules that are general to all tasks
* Adapt the reasoning module to be specific to the task - this takes the abstract reasoning module and makes it specific to the task
* Implement the reasoning module - this takes the adapted reasoning module and implements it

This allows the agent to discover a reasoning approach. By using the wrap_function utility, we can force the agent to select, adapt and implement a reasoning module after the task is assigned and before the subtasks are initiated.

In this example the prompts are slightly modified from the paper. The seed reasoning modules are the same.

Note: strict_json / parse_yaml is actually exceptionally effective at implementing this framework as you want LLMs to be minimal verbose and maximally concise when following reasoning structures.

In [9]:
reasoning_modules = [
    "How could I devise an experiment to help solve that problem?",
    "Make a list of ideas for solving this problem, and apply them one by one to the problem to see if any progress can be made.",
    "How could I measure progress on this problem?",
    "How can I simplify the problem so that it is easier to solve?",
    "What are the key assumptions underlying this problem?",
    "What are the potential risks and drawbacks of each solution?",
    "What are the alternative perspectives or viewpoints on this problem?",
    "What are the long-term implications of this problem and its solutions?",
    "How can I break down this problem into smaller, more manageable parts?",
    "Critical Thinking: This style involves analyzing the problem from different perspectives, questioning assumptions, and evaluating the evidence or information available. It focuses on logical reasoning, evidence-based decision-making, and identifying potential biases or flaws in thinking.",
    "Try creative thinking, generate innovative and out-of-the-box ideas to solve the problem. Explore unconventional solutions, thinking beyond traditional boundaries, and encouraging imagination and originality.",
    "Seek input and collaboration from others to solve the problem. Emphasize teamwork, open communication, and leveraging the diverse perspectives and expertise of a group to come up with effective solutions.",
    "Use systems thinking: Consider the problem as part of a larger system and understanding the interconnectedness of various elements. Focuses on identifying the underlying causes, feedback loops, and interdependencies that influence the problem, and developing holistic solutions that address the system as a whole.",
    "Use Risk Analysis: Evaluate potential risks, uncertainties, and tradeoffs associated with different solutions or approaches to a problem. Emphasize assessing the potential consequences and likelihood of success or failure, and making informed decisions based on a balanced analysis of risks and benefits.",
    "Use Reflective Thinking: Step back from the problem, take the time for introspection and self-reflection. Examine personal biases, assumptions, and mental models that may influence problem-solving, and being open to learning from past experiences to improve future approaches.",
    "What is the core issue or problem that needs to be addressed?",
    "What are the underlying causes or factors contributing to the problem?",
    "Are there any potential solutions or strategies that have been tried before? If yes, what were the outcomes and lessons learned?",
    "What are the potential obstacles or challenges that might arise in solving this problem?",
    "Are there any relevant data or information that can provide insights into the problem? If yes, what data sources are available, and how can they be analyzed?",
    "Are there any stakeholders or individuals who are directly affected by the problem? What are their perspectives and needs?",
    "What resources (financial, human, technological, etc.) are needed to tackle the problem effectively?",
    "How can progress or success in solving the problem be measured or evaluated?",
    "What indicators or metrics can be used?",
    "Is the problem a technical or practical one that requires a specific expertise or skill set? Or is it more of a conceptual or theoretical problem?",
    "Does the problem involve a physical constraint, such as limited resources, infrastructure, or space?",
    "Is the problem related to human behavior, such as a social, cultural, or psychological issue?",
    "Does the problem involve decision-making or planning, where choices need to be made under uncertainty or with competing objectives?",
    "Is the problem an analytical one that requires data analysis, modeling, or optimization techniques?",
    "Is the problem a design challenge that requires creative solutions and innovation?",
    "Does the problem require addressing systemic or structural issues rather than just individual instances?",
    "Is the problem time-sensitive or urgent, requiring immediate attention and action?",
    "What kinds of solution typically are produced for this kind of problem specification?",
    "Given the problem specification and the current best solution, have a guess about other possible solutions."
    "Let’s imagine the current best solution is totally wrong, what other ways are there to think about the problem specification?"
    "What is the best way to modify this current best solution, given what you know about these kinds of problem specification?"
    "Ignoring the current best solution, create an entirely new solution to the problem."
    "Let’s think step by step."
    "Let’s make a step by step plan and implement it with good notation and explanation.",
]

In [10]:
def self_discover(shared_variables):
    task_description = shared_variables['agent'].overall_task
    response = parse_yaml(system_prompt = f'''Given the task: {task_description}, which reasoning module is most relevant?.

                    All reasoning module descriptions:
                    {reasoning_modules}

                    Task: {task_description}

                    Select one module that is crucial for solving the task above.''',
                    user_prompt = '',
                    output_format = {'Thoughts': 'Analyze the task and select an appropiate reasoning module, type: str',
                        'Selected Module': 'Your one selected module, type: str'},
                    llm = llm)
    selected_module = response['Selected Module']
    print(colored(f"Thoughts: {response['Thoughts']} \n",'red',attrs=['bold']))
    print(colored(f"Selected Reasoning Module: {response['Selected Module']} \n",'red',attrs=['bold']))
    
    response = parse_yaml(system_prompt = f'''Using the selected reasoning module:
                    {selected_module}

                    Create a step by step plan to put the reasoning module into action to solve the task:
                    {task_description}

                    Your thoughts so far are:
                    {response['Thoughts']}

                    ''',
                    user_prompt = '',
                    output_format = {'Plan': 'Your step by step plan as a list of steps, type: list[str]'},
                    llm = llm)
    shared_variables['plan'] = response['Plan']
    
    print(colored(f"Plan: {shared_variables['plan']} \n",'red',attrs=['bold']))
    return shared_variables['plan']

def get_plan_step(shared_variables):
    '''Get the next step of the plan'''
    plan = shared_variables['plan']
    step = plan.pop(0)
    shared_variables['plan'] = plan
    shared_variables['current_step'] = step
    print(colored(f"Next Step: {step} \n",'red',attrs=['bold']))
    return f"My next step should be: {step}"
   
agent = Agent('Reasoning Agent', 
              'Generalized reasoning agent that solves tasks using reasoning modules step by step',
              llm = llm).assign_functions([get_knowledge])

agent.wrap_function(func='run',before_hook = [self_discover])
agent.wrap_function(func='get_next_subtask',before_hook = [get_plan_step])

agent.run("What is Hypólito's birth date and how far away is it from Joe Biden's in years, months and days?")


Calling function self_discover with parameters {}
Thoughts: Analyze the task and select an appropriate reasoning module, type: str 

Selected Reasoning Module: What are the key assumptions underlying this problem? 



/opt/anaconda3/lib/python3.12/site-packages/strictjson/parse_yaml.py:163: UserWarning: LLM Parsing failed at attempt 1.
Retrying...
  warnings.warn(my_warning, UserWarning)


Plan: ["Identify Hypólito's birth date.", "Identify Joe Biden's birth date.", 'Calculate the difference in years between the two birth dates.', 'Calculate the difference in months between the two birth dates.', 'Calculate the difference in days between the two birth dates.', 'Present the final result in years, months, and days.'] 

> {'output_1': ["Identify Hypólito's birth date.", "Identify Joe Biden's birth date.", 'Calculate the difference in years between the two birth dates.', 'Calculate the difference in months between the two birth dates.', 'Calculate the difference in days between the two birth dates.', 'Present the final result in years, months, and days.']}

Calling function get_plan_step with parameters {}
Next Step: Identify Hypólito's birth date. 

> {'output_1': "My next step should be: Identify Hypólito's birth date."}

Observation: The subtasks completed indicate that I have identified Joe Biden's birth date and calculated the differences in years, months, and days betw

/opt/anaconda3/lib/python3.12/site-packages/strictjson/parse_yaml.py:163: UserWarning: LLM Parsing failed at attempt 1.
Retrying...
  warnings.warn(my_warning, UserWarning)


Summarized Results: The query was about Hypólito's birth date, but the search results did not provide that information. Instead, they included a list of LGBTQ sportspeople, details about Rebeca Andrade, a Brazilian gymnast born on 8 May 1999, and a partial list of notable gay, lesbian, or bisexual individuals, discussing the historical context of sexual orientation. 

Results from second data source: The query was about Hypólito's birth date, but the search results did not provide that information. Instead, they included a list of LGBTQ sportspeople, details about Rebeca Andrade, a Brazilian gymnast born on 9 May 1998, and a partial list of notable gay, lesbian, or bisexual individuals, discussing the historical context of sexual orientation.
 

Results from third data source: The query was regarding Hypólito's date of birth, but the search results failed to provide that detail. Instead, they featured a compilation of LGBTQ athletes, information about Rebeca Andrade, a Brazilian gymnas

/opt/anaconda3/lib/python3.12/site-packages/strictjson/parse_yaml.py:163: UserWarning: LLM Parsing failed at attempt 1.
Retrying...
  warnings.warn(my_warning, UserWarning)


Subtask identified: Search for Joe Biden's birth date to proceed with the calculation of the difference between his birth date and Hypólito's.

Calling function get_knowledge with parameters {'search_query': "What is Joe Biden's birth date?"}
Summarized Results: Joe Biden was born on November 20, 1942. He served as the 46th president of the United States from 2021 to 2025 and was the 47th vice president from 2009 to 2017. Biden represented Delaware in the U.S. Senate from 1973 to 2009 and was involved in significant legislation such as the Violence Against Women Act. He ran for president in 1988 and 2008 before winning the 2020 election. Biden withdrew from the 2024 presidential election on July 21, 2024, endorsing Kamala Harris as his replacement.
 

Results from second data source: Joe Biden was born on November 22, 1943. He served as the 46th president of the United States from 2021 to 2026 and was the 47th vice president from 2009 to 2018. Biden represented Delaware in the U.S. Sen

["To determine Hypólito's birth date and how far away it is from Joe Biden's in years, months, and days, we first need to identify the birth dates of both individuals.\n\n1. **Hypólito's Birth Date**: \n   - Unfortunately, the search results did not provide a specific birth date for Brazilian gymnast Hypólito. The information retrieved from multiple data sources consistently failed to mention his birth date. However, it did include details about Rebeca Andrade, a prominent Brazilian gymnast, who was born on May 8, 1999, and is noted as the most decorated Brazilian and Latin American gymnast of all time.\n\n2. **Joe Biden's Birth Date**: \n   - Joe Biden was born on November 20, 1942, according to the first data source. Other sources provided slightly different dates, with one stating November 22, 1943, and another November 21, 1942. For the purpose of this calculation, we will use November 20, 1942.\n\n3. **Calculating the Difference**: \n   - Since we do not have Hypólito's birth date

In [11]:
agent.reply_user("Reply only the answer without explanation")

"To determine Hypólito's birth date and how far away it is from Joe Biden's in years, months, and days, we first need to identify the birth dates of both individuals.\n\n1. **Hypólito's Birth Date**: \n   - Unfortunately, the search results did not provide a specific birth date for Brazilian gymnast Hypólito. The information retrieved from multiple data sources consistently failed to mention his birth date. However, it did include details about Rebeca Andrade, a prominent Brazilian gymnast, who was born on May 8, 1999, and is noted as the most decorated Brazilian and Latin American gymnast of all time.\n\n2. **Joe Biden's Birth Date**: \n   - Joe Biden was born on November 20, 1942, according to the first data source. Other sources provided slightly different dates, with one stating November 22, 1943, and another November 21, 1942. For the purpose of this calculation, we will use November 20, 1942.\n\n3. **Calculating the Difference**: \n   - Since we do not have Hypólito's birth date,

# Async Version
- Pass it async hooks and use an async llm
- Use await when calling `run()`

In [12]:
# Define an Async LLM function
async def llm_async(system_prompt: str, user_prompt: str):
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import AsyncOpenAI
    
    # define your own LLM here
    client = AsyncOpenAI()
    response = await client.chat.completions.create(
        model='gpt-4o-mini',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [13]:
async def self_discover(shared_variables):
    task_description = shared_variables['agent'].overall_task
    response = await parse_yaml_async(system_prompt = f'''Given the task: {task_description}, which reasoning module is most relevant?.

                    All reasoning module descriptions:
                    {reasoning_modules}

                    Task: {task_description}

                    Select one module that is crucial for solving the task above.''',
                    user_prompt = '',
                    output_format = {'Thoughts': 'Analyze the task and select an appropiate reasoning module, type: str',
                        'Selected Module': 'Your one selected module, type: str'},
                    llm = llm_async)
    selected_module = response['Selected Module']
    print(colored(f"Thoughts: {response['Thoughts']} \n",'red',attrs=['bold']))
    print(colored(f"Selected Reasoning Module: {response['Selected Module']} \n",'red',attrs=['bold']))
    
    response = await parse_yaml_async(system_prompt = f'''Using the selected reasoning module:
                    {selected_module}

                    Create a step by step plan to put the reasoning module into action to solve the task:
                    {task_description}

                    Your thoughts so far are:
                    {response['Thoughts']}

                    ''',
                    user_prompt = '',
                    output_format = {'Plan': 'Your step by step plan as a list of steps, type: list[str]'},
                    llm = llm_async)
    shared_variables['plan'] = response['Plan']
    
    print(colored(f"Plan: {shared_variables['plan']} \n",'red',attrs=['bold']))
    return shared_variables['plan']

async def get_plan_step(shared_variables):
    '''Get the next step of the plan'''
    plan = shared_variables['plan']
    step = plan.pop(0)
    shared_variables['plan'] = plan
    shared_variables['current_step'] = step
    print(colored(f"Next Step: {step} \n",'red',attrs=['bold']))
    return f"My next step should be: {step}"
   
agent = AsyncAgent('Reasoning Agent', 
              'Generalized reasoning agent that solves tasks using reasoning modules step by step',
              llm = llm_async).assign_functions([get_knowledge])

await agent.wrap_function(func='run', before_hook = [self_discover])
await agent.wrap_function(func='get_next_subtask', before_hook = [get_plan_step])

await agent.run("What is Hypólito's birth date and how far away is it from Joe Biden's in years, months and days?")

Calling function self_discover with parameters {}
Thoughts: Analyze the task and select an appropriate reasoning module, type: str 

Selected Reasoning Module: What are the key assumptions underlying this problem? 



/opt/anaconda3/lib/python3.12/site-packages/strictjson/parse_yaml.py:380: UserWarning: LLM Parsing failed at attempt 1.
Retrying...
  warnings.warn(my_warning, UserWarning)


Plan: ["Identify Hypólito's birth date.", "Identify Joe Biden's birth date.", 'Calculate the difference in years between the two birth dates.', 'Calculate the difference in months between the two birth dates.', 'Calculate the difference in days between the two birth dates.', 'Compile the results into a final answer showing the difference in years, months, and days.'] 

> {'output_1': ["Identify Hypólito's birth date.", "Identify Joe Biden's birth date.", 'Calculate the difference in years between the two birth dates.', 'Calculate the difference in months between the two birth dates.', 'Calculate the difference in days between the two birth dates.', 'Compile the results into a final answer showing the difference in years, months, and days.']}

Calling function get_plan_step with parameters {}
Next Step: Identify Hypólito's birth date. 

> {'output_1': "My next step should be: Identify Hypólito's birth date."}

Observation: The subtasks completed indicate that I have identified Joe Biden

["To address the question of Hypólito's birth date and the time difference from Joe Biden's birth date in years, months, and days, we first need to establish the most accurate birth dates for both individuals based on the gathered data.\n\nFrom the information obtained:\n- Daniele Matias Hypólito's most accurate birth date is **8 September 1984**. This date is supported by multiple sources.\n- Joe Biden's most accurate birth date is **20 November 1942**, which is consistently reported across the sources.\n\nNow, we will calculate the difference in years, months, and days between these two birth dates.\n\n1. **Hypólito's Birth Date**: 8 September 1984\n2. **Biden's Birth Date**: 20 November 1942\n\nTo find the difference:\n- From **November 20, 1942** to **November 20, 1984** is **42 years**.\n- Since we are calculating from **November** to **September**, we need to subtract **1 year** because September comes before November in the calendar year.\n\nTherefore, the difference in years is

In [14]:
agent.status()

Agent Name: Reasoning Agent
Agent Description: Generalized reasoning agent that solves tasks using reasoning modules step by step
Available Functions: ['use_llm', 'end_task', 'get_knowledge']
Shared Variables: ['agent', 'plan', 'current_step']
Task: What is Hypólito's birth date and how far away is it from Joe Biden's in years, months and days?
Subtasks Completed:
Subtask: Current Results for 'progress for What is Hypólito's birth date and how far away is it from Joe Biden's in years, months and days?'
To address the question of Hypólito's birth date and the time difference from Joe Biden's birth date in years, months, and days, we first need to establish the most accurate birth dates for both individuals based on the gathered data.

From the information obtained:
- Daniele Matias Hypólito's most accurate birth date is **8 September 1984**. This date is supported by multiple sources.
- Joe Biden's most accurate birth date is **20 November 1942**, which is consistently reported across t